## Notebook to clean and extract data for analysis/ML.

### Contents
1. Import required libraries
2. Read data for cleaning
3. Handle null rows
4. Clean data formatting for consistency
5. Handle and drop outliers 
6. Processing and Extraction
7. Join additional data for public holidays
8. Get and Join the daily weather data
9. Get All Station Name and Positions
10. Export data

### 1. Import required libraries

In [2]:
# Import libraries
import pandas as pd # for data manipulation and analysis
import numpy as np # for working with arrays and carrying out mathematical operations
import matplotlib.pyplot as plt # for plotting
import re # In-built regular expressions library

### 2. Read data for cleaning

In [3]:
# Import all the monthly csv dataset
dec_20 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202012-divvy-tripdata.csv')
jan_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202101-divvy-tripdata.csv')
feb_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202102-divvy-tripdata.csv')
march_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202103-divvy-tripdata.csv')
april_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202104-divvy-tripdata.csv')
may_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202105-divvy-tripdata.csv')
jun_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202106-divvy-tripdata.csv')
july_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202107-divvy-tripdata.csv')
aug_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202108-divvy-tripdata.csv')
sept_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202109-divvy-tripdata.csv')
oct_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202110-divvy-tripdata.csv')
nov_21 = pd.read_csv(r'/Users/Razak/Documents/Cyclistic Project/Data/202111-divvy-tripdata.csv')


# Join all months into a single dataframe
months = [dec_20, jan_21, feb_21, march_21, april_21, may_21, jun_21, july_21,
           aug_21, sept_21, oct_21, nov_21]
df_months = pd.concat(months)

# Check the number of data points
print(f'Number of data points: {df_months.shape[0]}')

Number of data points: 5479096


In [4]:
# Drop duplicates
df = df_months
print('Number of duplicate ride_ids:',df['ride_id'].duplicated().sum())

df = df.drop_duplicates(subset=['ride_id'])
print('No. duplicates after dropping:', df['ride_id'].duplicated().sum())

Number of duplicate ride_ids: 0
No. duplicates after dropping: 0


### 3. Handle null rows

In [5]:
# Drop rows without start and end time information
df = df.dropna(subset= ['started_at','ended_at'])

df = df.sort_values(by=['start_lat','start_lng'], ascending=True)

# Forward fill the missing start_station_name and start_station_id
df[['start_station_name', 'start_station_id']] = df[['start_station_name', 'start_station_id']].ffill().bfill()

# Fill the end stations
df = df.sort_values(by=['end_lat','end_lng'], ascending=True)
# Forward fill the missing start_station_name and start_station_id
df[['end_station_name', 'end_station_id','end_lat','end_lng']] = df[['end_station_name', 'end_station_id','end_lat','end_lng']].ffill().bfill()

df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

### 4. Clean data formatting for consistency

In [6]:
# Data Extraction
df_clean = df

# Funtion to preprocess datetime string, adding seconds if missing
def preprocess_datetime(dt_str):
    if pd.isna(dt_str):
        return np.nan
    if len(dt_str) == 16:  # Format is missing seconds (YYYY-MM-DD HH:MM)
        return dt_str + ':00'
    # Add leading zero to single-digit hours
    if len(dt_str) == 15 and re.match(r'\d{4}-\d{2}-\d{2} \d:\d{2}', dt_str):
        return dt_str[:11] + '0' + dt_str[11:] + ':00'
    
    # Add leading zero to single-digit hours with missing seconds
    if len(dt_str) == 14 and re.match(r'\d{4}-\d{2}-\d{2} \d:\d{2}', dt_str):
        return dt_str[:11] + '0' + dt_str[11:] + ':00'
    return dt_str

# Apply the preprocessing function to both columns
df_clean['started_at'] = df_clean['started_at'].apply(preprocess_datetime)
df_clean['ended_at'] = df_clean['ended_at'].apply(preprocess_datetime)

# Convert to date time
df_clean['started_at'] = pd.to_datetime(df_clean['started_at'],format='%Y-%m-%d %H:%M:%S')
df_clean['ended_at'] = pd.to_datetime(df_clean['ended_at'],format='%Y-%m-%d %H:%M:%S')
# Extract the month and day of the week
df['day'] = df['started_at'].dt.day_name()
df['month'] = df['started_at'].dt.strftime("%B")

# Find the ride duration in seconds
df_clean['ride_duration'] = (df_clean['ended_at']-df_clean['started_at']).dt.total_seconds()

### 5. Handle and drop outliers 

In [7]:
# Remove outliers
p25 = df_clean['ride_duration'].quantile(0.25) # 25th percentile
p50 = df_clean['ride_duration'].quantile(0.5) # 50th percentile
p75 = df_clean['ride_duration'].quantile(0.75) # 75th percentile

# Interquartile range
iqr = p75 - p25
print(f'Interquartile range: {iqr}')

# Lower Limit
lower_bound  = p25 - 1.5 * iqr
print(f'Lower Bound: {lower_bound}')

# Upper Limit
upper_bound = p75 + 1.5 * iqr
print(f'Upper Bound: {upper_bound}')

# Mean
print(f'Mean: {df_clean['ride_duration'].mean()}')

# Eliminate outliers above the upper bound and negative ride durations
df_clean = df_clean[(df_clean['ride_duration'] >= 0) & (df_clean['ride_duration'] <= upper_bound)]

print(f'No of data points after cleaning: {df_clean.shape[0]}')

Interquartile range: 910.0
Lower Bound: -955.0
Upper Bound: 2685.0
Mean: 1209.2180638922916
No of data points after cleaning: 5072961


### 6. Processing and Extraction

In [ ]:
# Extract the hour of the day
df_clean['hour'] = df_clean['started_at'].dt.hour # extract the ride hour

# function to assign time of day
def time_of_day(row):
    if 0 <= row['hour'] < 6:
        return 'Early Morning'
    elif 6 <= row['hour'] < 12:
        return 'Morning'
    elif 12 <= row['hour'] < 18:
        return 'Afternoon'
    elif 18 <= row['hour'] < 21:
        return 'Evening'
    elif 21 <= row['hour'] < 24:
        return 'Night'
    
df_clean['time_of_day'] = df_clean.apply(time_of_day, axis = 1)

In [ ]:
# Function to assign seasons based on the month
def assign_season(row):
    if row['month'] in ['December', 'January', 'February']:
        return 'Winter'
    elif row['month'] in ['March', 'April', 'May']:
        return 'Spring'
    elif row['month'] in ['June', 'July', 'August']:
        return 'Summer'
    elif row['month'] in ['September', 'October', 'November']:
        return 'Fall'
    
# Apply the function to assign seasons
df_clean['season'] = df_clean.apply(assign_season, axis=1)

### 7. Join additional data for public holidays
Here's my approach to achieve this:
1. Get all public holidays in Chicago that year (2021) via web scraping
2. Extract the date from the 'started_at' column
3. Join with the main dataframe

In [ ]:
url = 'https://www.timeanddate.com/holidays/us/2021'

# Scrape the raw table
holiday_data = pd.read_html(url)[0]

# Drop first levl column of the multi-index
holiday_data.columns = holiday_data.columns.droplevel(0)

#renam column
df_holiday = holiday_data.rename(columns={'Unnamed: 1_level_1': 'Day'})

# Select needed columns and rows
holidays = df_holiday[['Date','Day','Name']].iloc[1:-1].reset_index(drop=True)

# Extract the date

df_clean['start_date'] = df_clean['started_at'].dt.strftime("%b %d")
df_clean['start_day'] = df_clean['started_at'].dt.strftime("%Y-%m-%d")

# Join 
holiday_df = pd.merge(df_clean, holidays, left_on ='start_date', right_on = 'Date', how = 'left').drop_duplicates(subset = 'ride_id')

# get holiday columns
def holiday(row):
    if pd.isna(row['Name']):
        return 'Not Holiday'
    else:
        return 'Holiday'
    
holiday_df['holiday'] = holiday_df.apply(holiday, axis = 1)

### 8. Get and Join the daily weather data

In [11]:
# import climate data
weather_df = pd.read_csv(r'./Chicago,United States 2020-12-01 to 2021-12-01.csv')

# Clean data
# Convert the 'datetime' column to actual datetime objects
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], format='%Y-%m-%dT%H:%M:%S')
weather_df['datetime'] = weather_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_weather = weather_df[['datetime','conditions','icon','temp','humidity','windspeed']]

holiday_df['start_date_hour'] = holiday_df['started_at'].apply(lambda x: x.replace(minute=0, second=0))
# Optionally convert it back to string format
holiday_df['start_date_hour'] = holiday_df['start_date_hour'].dt.strftime("%Y-%m-%d %H:%M:%S")

full_df = pd.merge(holiday_df, df_weather, left_on='start_date_hour', right_on = 'datetime', how ='left').drop_duplicates(subset = 'ride_id')

# function to group the temperature into  (warm, cold, hot, cool)
def categorize_temperature(row):
    if row['temp'] < 10:
        return 'Cold'
    elif 10 <= row['temp'] < 20:
        return 'Cool'
    elif 20 <= row['temp'] < 30:
        return 'Warm'
    else:
        return 'Hot'
    
# Apply the function
full_df['temp_feel'] = full_df.apply(categorize_temperature, axis = 1)

# Preview data
full_df.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,Name,holiday,start_date_hour,datetime,conditions,icon,temp,humidity,windspeed,temp_feel
0,097BB436A70E5964,electric_bike,2021-02-27 01:27:00,2021-02-27 02:09:00,Indiana Ave & 133rd St,20240,Hegewisch Metra Station,20215,41.65,-87.54,...,NaN,Not Holiday,2021-02-27 01:00:00,2021-02-27 01:00:00,"Rain, Overcast",rain,2.4,81.47,10.1,Cold
1,401BC56B23FA4A26,electric_bike,2021-10-08 12:12:05,2021-10-08 12:35:25,Major Taylor Trail & 124th St,20208,Hegewisch Metra Station,20215,41.67,-87.55,...,NaN,Not Holiday,2021-10-08 12:00:00,2021-10-08 12:00:00,Partially cloudy,partly-cloudy-day,22.0,66.12,11.0,Warm


### 9. Get All Station Name and Positions

In [ ]:
start_stations =full_df[['start_station_name',
'start_lat','start_lng']].drop_duplicates('start_station_name').reset_index(drop=True)

# end station names
end_stations = full_df[['end_station_name',
'end_lat','end_lng']].drop_duplicates('end_station_name').reset_index(drop=True)

# rename columns
start_stations.rename(columns = {'start_station_name':'station_name',
                                'start_lat':'latitude', 'start_lng':'longitude'}, inplace =True)

end_stations.rename(columns = {'end_station_name':'station_name',
                                'end_lat':'latitude', 'end_lng':'longitude'}, inplace =True)

# Join the start and end stations into a single dataframe              
stations = pd.concat([start_stations, end_stations], axis=0)

# group to get unique lat and long for each station
station_df = stations.groupby('station_name').agg({'latitude':'mean', 'longitude':'mean'}).reset_index()

no_start_rides = full_df.groupby('start_station_name').agg({'ride_id':'count'}).sort_values(by='ride_id', ascending=False).reset_index().rename(columns={'ride_id':'start_trips'})
no_end_rides = full_df.groupby('end_station_name').agg({'ride_id':'count'}).sort_values(by='ride_id', ascending=False).reset_index().rename(columns={'ride_id':'end_trips'})

total_trips = no_start_rides.merge(no_end_rides, left_on='start_station_name', right_on='end_station_name', how = 'outer')

# Fill NaN values with 0 (in case a station only has start or end trips)
total_trips['start_trips'].fillna(0, inplace=True)
total_trips['end_trips'].fillna(0, inplace=True)

# Rename and reorder columns for clarity
total_trips = total_trips.rename(columns={'start_station_name': 'station_name'})
total_trips = total_trips[['station_name', 'start_trips', 'end_trips']]

total_trips['station_usage'] = total_trips['start_trips'] + total_trips['end_trips']
total_trips['station_name'].fillna('California Ave & 29th St', inplace=True)

df_station = station_df.merge(total_trips, on='station_name').drop(columns=['start_trips','end_trips'], axis=1)
df_station['station_usage'] = df_station['station_usage'].astype(int)

df_station.sort_values(by='station_usage', ascending=False).head()

,station_name,latitude,longitude,station_usage
736,Streeter Dr & Grand Ave,41.849948,-87.599118,125621
637,Rockwell St & 57th St,41.790000,-87.645000,90366
787,Wells St & Concord Ln,41.896022,-87.635199,83349
790,Wells St & Hubbard St,41.887114,-87.629962,81369
153,Clark St & Elm St,41.886277,-87.624250,77987


### 10. Get Start Station Traffic

In [60]:
def traffic(row, q25, q50):
    if row['station_usage'] <= q25:
        return 0
    elif q25 < row['station_usage'] <= q50:
        return 1
    else:
        return 2

# Calculate quantiles for 'station_usage' column
q25 = df_station['station_usage'].quantile(0.25)
q50 = df_station['station_usage'].quantile(0.50)

# Apply the function, passing the quantiles as additional arguments
df_station['station_traffic'] = df_station.apply(traffic, axis=1, q25=q25, q50=q50)

# Display the updated DataFrame
df_station.head()


,station_name,latitude,longitude,station_usage,station_traffic
0,2112 W Peterson Ave,41.972080,-87.715024,2066,1
1,351,41.930000,-87.780000,2,0
2,63rd St Beach,41.778329,-87.574033,2945,1
3,900 W Harrison St,41.855536,-87.628546,15828,2
4,Aberdeen St & Jackson Blvd,41.872538,-87.655750,22954,2


In [63]:
df_station = df_station.rename(columns={'station_name': 'start_station_name'})

# Merge the dataframes on 'start_station_name'
df_merged = full_df.merge(df_station[['start_station_name', 'station_traffic']], 
                           on='start_station_name', 
                           how='left')

# Display the merged dataframe
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,holiday,start_date_hour,datetime,conditions,icon,temp,humidity,windspeed,temp_feel,station_traffic
0,097BB436A70E5964,electric_bike,2021-02-27 01:27:00,2021-02-27 02:09:00,Indiana Ave & 133rd St,20240,Hegewisch Metra Station,20215,41.650000,-87.540000,...,Not Holiday,2021-02-27 01:00:00,2021-02-27 01:00:00,"Rain, Overcast",rain,2.4,81.47,10.1,Cold,0
1,401BC56B23FA4A26,electric_bike,2021-10-08 12:12:05,2021-10-08 12:35:25,Major Taylor Trail & 124th St,20208,Hegewisch Metra Station,20215,41.670000,-87.550000,...,Not Holiday,2021-10-08 12:00:00,2021-10-08 12:00:00,Partially cloudy,partly-cloudy-day,22.0,66.12,11.0,Warm,1
2,A47B92B608C6BFD9,electric_bike,2021-08-28 15:09:09,2021-08-28 15:53:14,State St & 123rd St,20210,Hegewisch Metra Station,20215,41.680000,-87.640000,...,Not Holiday,2021-08-28 15:00:00,2021-08-28 15:00:00,Partially cloudy,partly-cloudy-day,33.2,51.89,15.3,Hot,1
3,5F7DF59888132DAA,electric_bike,2021-11-21 12:43:04,2021-11-21 13:16:09,State St & 95th St,20104,Hegewisch Metra Station,20215,41.721652,-87.622815,...,Holiday,2021-11-21 12:00:00,2021-11-21 12:00:00,Partially cloudy,partly-cloudy-day,9.9,55.27,24.8,Cold,1
4,31E60C9C43FF7BAA,electric_bike,2021-09-23 22:23:34,2021-09-23 22:46:35,Commercial Ave & 130th St,20213,Hegewisch Metra Station,20215,41.660000,-87.650000,...,Holiday,2021-09-23 22:00:00,2021-09-23 22:00:00,Partially cloudy,partly-cloudy-night,13.9,54.52,10.8,Cool,1


### 10. Export data

In [66]:
df_merged.to_csv(r'./Cleaned Dataframe.csv', index=False)
df_station.to_csv(r'./Station Names.csv', index=False)

Sources:
- Weather: 'https://www.visualcrossing.com/weather/weather-data-services/Chicago,United%20States/metric/2020-12-01/2021-11-30'
- Holidays: 'https://www.timeanddate.com/weather/usa/chicago/historic?month=1&year=2021'